In [1]:
import pandas as pd
import numpy as np

# 数据集处理

In [2]:
from util.data_loader import data_loader2
data_name='bank_data_B'
miss_rate=0.5
complete_data_B,incomplete_data_B=data_loader2(data_name,miss_rate)
print(complete_data_B.shape)
print(incomplete_data_B.shape)

(45211, 10)
(22605, 10)


# CTGAN生成

In [3]:
discrete_columns=incomplete_data_B.select_dtypes(include=['object'])
from ctgan import CTGAN
ctgan = CTGAN(epochs=10)
ctgan.fit(incomplete_data_B,discrete_columns)
len1=complete_data_B.shape[0]
# 生成数据
synthetic_data = ctgan.sample(len1)

D:\Anaconda\anaconda\envs\neural_network\lib\site-packages\rdt\transformers\base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
D:\Anaconda\anaconda\envs\neural_network\lib\site-packages\rdt\transformers\base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
D:\Anaconda\anaconda\envs\neural_network\lib\site-packages\rdt\transformers\base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
D:\Anaconda\anaconda\envs\neural_network\lib\site-packages\rdt\transformers\base.py:132: FutureWarning: Future versions of RDT will not 

# 得到A方完整数据和B方生成后的数据集

In [4]:
complete_data_A = pd.read_csv('datasetsAB/bank_data_A.csv', sep=',')
# 假设A和B是你的DataFrame，你想保留A中的重复列
aligned_df = complete_data_A.merge(incomplete_data_B, left_index=True, right_index=True, how='outer', suffixes=('_keep', '_drop'))

# 删除B中的重复列
columns_to_drop = [column for column in aligned_df.columns if '_drop' in column]
aligned_df = aligned_df.drop(columns_to_drop, axis=1)

# 删除列名中的后缀
aligned_df.columns = aligned_df.columns.str.replace('_keep', '')
print(aligned_df.shape)
aligned_df.head(50)

(45211, 18)


,age,job,marital,education,default,balance,housing,loan,index,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,0,unknown,5.0,may,261.0,1.0,-1.0,0.0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,1,unknown,5.0,may,151.0,1.0,-1.0,0.0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,2,unknown,5.0,may,76.0,1.0,-1.0,0.0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,33,unknown,single,unknown,no,1,no,no,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,35,management,married,tertiary,no,231,yes,no,5,unknown,5.0,may,139.0,1.0,-1.0,0.0,unknown,no
6,28,management,single,tertiary,no,447,yes,yes,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,42,entrepreneur,divorced,tertiary,yes,2,yes,no,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,58,retired,married,primary,no,121,yes,no,8,unknown,5.0,may,50.0,1.0,-1.0,0.0,unknown,no
9,43,technician,single,secondary,no,593,yes,no,9,unknown,5.0,may,55.0,1.0,-1.0,0.0,unknown,no


In [5]:
# data_m = 1 - np.isnan(aligned_df.values)  # 创建掩模矩阵，标记缺失值位置
# 创建掩码矩阵
data_m = aligned_df.notnull().astype(int)
data_m

,age,job,marital,education,default,balance,housing,loan,index,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0
4,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0
45207,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
45208,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0
45209,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0


In [6]:
# 将生成的数据集与原有的数据集合并
# 假设df1和df2是你的两个DataFrame
aligned_df_new = aligned_df.combine_first(synthetic_data)
print(aligned_df_new.shape)
aligned_df_new.head(50)

(45211, 18)


,age,balance,campaign,contact,day,default,duration,education,housing,index,job,loan,marital,month,pdays,poutcome,previous,y
0,58,2143,1.0,unknown,5.0,no,261.0,tertiary,yes,0,management,no,married,may,-1.0,unknown,0.0,no
1,44,29,1.0,unknown,5.0,no,151.0,secondary,yes,1,technician,no,single,may,-1.0,unknown,0.0,no
2,33,2,1.0,unknown,5.0,no,76.0,secondary,yes,2,entrepreneur,yes,married,may,-1.0,unknown,0.0,no
3,47,1506,1.0,cellular,12.0,no,439.0,unknown,yes,3,blue-collar,no,married,jun,175.0,unknown,0.0,no
4,33,1,2.0,cellular,17.0,no,354.0,unknown,no,4,unknown,no,single,may,-2.0,unknown,1.0,no
5,35,231,1.0,unknown,5.0,no,139.0,tertiary,yes,5,management,no,married,may,-1.0,unknown,0.0,no
6,28,447,1.0,cellular,17.0,no,1026.0,tertiary,yes,6,management,yes,single,may,162.0,unknown,0.0,no
7,42,2,2.0,cellular,20.0,yes,67.0,tertiary,yes,7,entrepreneur,no,divorced,jun,-2.0,failure,0.0,no
8,58,121,1.0,unknown,5.0,no,50.0,primary,yes,8,retired,no,married,may,-1.0,unknown,0.0,no
9,43,593,1.0,unknown,5.0,no,55.0,secondary,yes,9,technician,no,single,may,-1.0,unknown,0.0,no


# 获得原始数据集

In [7]:
# 假设bank_data是原始数据集
bank_data = pd.read_csv('datasets/bank.csv',sep=';')
bank_data['index']=range(0,len(bank_data))
print(bank_data.shape)
bank_data.head(10)

(45211, 18)


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,index
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no,0
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no,1
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no,2
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no,3
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no,4
5,35,management,married,tertiary,no,231,yes,no,unknown,5,may,139,1,-1,0,unknown,no,5
6,28,management,single,tertiary,no,447,yes,yes,unknown,5,may,217,1,-1,0,unknown,no,6
7,42,entrepreneur,divorced,tertiary,yes,2,yes,no,unknown,5,may,380,1,-1,0,unknown,no,7
8,58,retired,married,primary,no,121,yes,no,unknown,5,may,50,1,-1,0,unknown,no,8
9,43,technician,single,secondary,no,593,yes,no,unknown,5,may,55,1,-1,0,unknown,no,9


In [8]:
# 将bank_data_encode数据集的列的顺序调整到和aligned_df_new_encode列的顺序一样
aligned_df_new = aligned_df_new.reindex(columns=bank_data.columns)
aligned_df_new.head(50)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,index
0,58,management,married,tertiary,no,2143,yes,no,unknown,5.0,may,261.0,1.0,-1.0,0.0,unknown,no,0
1,44,technician,single,secondary,no,29,yes,no,unknown,5.0,may,151.0,1.0,-1.0,0.0,unknown,no,1
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5.0,may,76.0,1.0,-1.0,0.0,unknown,no,2
3,47,blue-collar,married,unknown,no,1506,yes,no,cellular,12.0,jun,439.0,1.0,175.0,0.0,unknown,no,3
4,33,unknown,single,unknown,no,1,no,no,cellular,17.0,may,354.0,2.0,-2.0,1.0,unknown,no,4
5,35,management,married,tertiary,no,231,yes,no,unknown,5.0,may,139.0,1.0,-1.0,0.0,unknown,no,5
6,28,management,single,tertiary,no,447,yes,yes,cellular,17.0,may,1026.0,1.0,162.0,0.0,unknown,no,6
7,42,entrepreneur,divorced,tertiary,yes,2,yes,no,cellular,20.0,jun,67.0,2.0,-2.0,0.0,failure,no,7
8,58,retired,married,primary,no,121,yes,no,unknown,5.0,may,50.0,1.0,-1.0,0.0,unknown,no,8
9,43,technician,single,secondary,no,593,yes,no,unknown,5.0,may,55.0,1.0,-1.0,0.0,unknown,no,9


In [9]:
dtypes=aligned_df_new.dtypes
print(dtypes)

age            int64
job           object
marital       object
education     object
default       object
balance        int64
housing       object
loan          object
contact       object
day          float64
month         object
duration     float64
campaign     float64
pdays        float64
previous     float64
poutcome      object
y             object
index          int64
dtype: object


In [10]:
bank_data=bank_data.astype(dtypes)
print(bank_data.dtypes)

age            int64
job           object
marital       object
education     object
default       object
balance        int64
housing       object
loan          object
contact       object
day          float64
month         object
duration     float64
campaign     float64
pdays        float64
previous     float64
poutcome      object
y             object
index          int64
dtype: object


In [11]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

# 使用fit_transform方法对标签列进行标签编码
bank_data['y'] = le.fit_transform(bank_data['y'])
aligned_df_new['y'] = le.fit_transform(aligned_df_new['y'])

In [12]:
# 获得分类列列名
discrete_columns=bank_data.select_dtypes(include=['object']).columns
discrete_columns

Index(['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact',
       'month', 'poutcome'],
      dtype='object')

In [13]:
bank_data

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,index
0,58,management,married,tertiary,no,2143,yes,no,unknown,5.0,may,261.0,1.0,-1.0,0.0,unknown,0,0
1,44,technician,single,secondary,no,29,yes,no,unknown,5.0,may,151.0,1.0,-1.0,0.0,unknown,0,1
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5.0,may,76.0,1.0,-1.0,0.0,unknown,0,2
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5.0,may,92.0,1.0,-1.0,0.0,unknown,0,3
4,33,unknown,single,unknown,no,1,no,no,unknown,5.0,may,198.0,1.0,-1.0,0.0,unknown,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17.0,nov,977.0,3.0,-1.0,0.0,unknown,1,45206
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17.0,nov,456.0,2.0,-1.0,0.0,unknown,1,45207
45208,72,retired,married,secondary,no,5715,no,no,cellular,17.0,nov,1127.0,5.0,184.0,3.0,success,1,45208
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17.0,nov,508.0,4.0,-1.0,0.0,unknown,0,45209


In [14]:
bank_data_encode=bank_data.copy()
# 对每一列进行检查
for col in bank_data_encode.columns:
    # 如果该列是object类型（通常意味着它是分类的），则进行标签编码
    if bank_data_encode[col].dtype == 'object':

        bank_data_encode[col] =le.fit_transform(bank_data_encode[col])

In [15]:
aligned_df_new_encode=aligned_df_new.copy()
# 对每一列进行检查
for col in aligned_df_new_encode.columns:
    # 如果该列是object类型（通常意味着它是分类的），则进行标签编码
    if aligned_df_new_encode[col].dtype == 'object':

        aligned_df_new_encode[col] =le.fit_transform(aligned_df_new_encode[col])

In [16]:
aligned_df_new_encode

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,index
0,58,4,1,2,0,2143,1,0,2,5.0,8,261.0,1.0,-1.0,0.0,3,0,0
1,44,9,2,1,0,29,1,0,2,5.0,8,151.0,1.0,-1.0,0.0,3,0,1
2,33,2,1,1,0,2,1,1,2,5.0,8,76.0,1.0,-1.0,0.0,3,0,2
3,47,1,1,3,0,1506,1,0,0,12.0,6,439.0,1.0,175.0,0.0,3,0,3
4,33,11,2,3,0,1,0,0,0,17.0,8,354.0,2.0,-2.0,1.0,3,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,9,1,2,0,825,0,0,2,5.0,4,343.0,1.0,77.0,1.0,3,0,45206
45207,71,5,0,0,0,1729,0,0,0,17.0,9,456.0,2.0,-1.0,0.0,3,1,45207
45208,72,5,1,1,0,5715,0,0,0,18.0,3,309.0,1.0,102.0,0.0,3,1,45208
45209,57,1,1,1,0,668,0,0,0,31.0,9,155.0,5.0,1.0,2.0,2,0,45209


In [17]:
bank_data_encode

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,index
0,58,4,1,2,0,2143,1,0,2,5.0,8,261.0,1.0,-1.0,0.0,3,0,0
1,44,9,2,1,0,29,1,0,2,5.0,8,151.0,1.0,-1.0,0.0,3,0,1
2,33,2,1,1,0,2,1,1,2,5.0,8,76.0,1.0,-1.0,0.0,3,0,2
3,47,1,1,3,0,1506,1,0,2,5.0,8,92.0,1.0,-1.0,0.0,3,0,3
4,33,11,2,3,0,1,0,0,2,5.0,8,198.0,1.0,-1.0,0.0,3,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,9,1,2,0,825,0,0,0,17.0,9,977.0,3.0,-1.0,0.0,3,1,45206
45207,71,5,0,0,0,1729,0,0,0,17.0,9,456.0,2.0,-1.0,0.0,3,1,45207
45208,72,5,1,1,0,5715,0,0,0,17.0,9,1127.0,5.0,184.0,3.0,2,1,45208
45209,57,1,1,1,0,668,0,0,1,17.0,9,508.0,4.0,-1.0,0.0,3,0,45209


In [18]:
bank_data_encode=bank_data_encode.astype(float)
print(bank_data_encode.dtypes)

age          float64
job          float64
marital      float64
education    float64
default      float64
balance      float64
housing      float64
loan         float64
contact      float64
day          float64
month        float64
duration     float64
campaign     float64
pdays        float64
previous     float64
poutcome     float64
y            float64
index        float64
dtype: object


In [19]:
aligned_df_new_encode=aligned_df_new_encode.astype(float)
print(aligned_df_new_encode.dtypes)

age          float64
job          float64
marital      float64
education    float64
default      float64
balance      float64
housing      float64
loan         float64
contact      float64
day          float64
month        float64
duration     float64
campaign     float64
pdays        float64
previous     float64
poutcome     float64
y            float64
index        float64
dtype: object


In [21]:
from table_evaluator import TableEvaluator
# 传入是数据类型得是DataForm类型
evaluator=TableEvaluator(bank_data_encode,aligned_df_new_encode)
# 数值分析
evaluator.evaluate(target_col='y',notebook=False,verbose=False)


Classifier F1-scores and their Jaccard similarities::
                             f1_real  f1_fake  jaccard_similarity
index                                                            
DecisionTreeClassifier_fake   0.7991   0.7439              0.5918
DecisionTreeClassifier_real   0.8916   0.9317              0.7976
LogisticRegression_fake       0.7911   0.8096              0.8887
LogisticRegression_real       0.8924   0.8874              0.9159
MLPClassifier_fake            0.7556   0.8080              0.7051
MLPClassifier_real            0.8728   0.8850              0.7581
RandomForestClassifier_fake   0.8332   0.8130              0.7868
RandomForestClassifier_real   0.9068   0.9348              0.8672

Privacy results:
                                         result
Duplicate rows between sets (real/fake)  (0, 0)
nearest neighbor mean                    1.0038
nearest neighbor std                     0.6351

Miscellaneous results:
                                  Result
Column Cor

# 对B方进行编码

In [22]:
aligned_df_new_encode=aligned_df_new.copy()
aligned_df_new_encode.head(50)
# 数据预处理
# 对分类变量进行one-hot编码
aligned_df_new_encode = pd.get_dummies(aligned_df_new_encode, drop_first=True)
# 定义特征和目标变量
X = aligned_df_new_encode.drop('y', axis=1)  # 假设 '' 是目标变量
y = aligned_df_new_encode['y']
print(X.shape)
print(y.shape)

(45211, 43)
(45211,)


In [23]:
from sklearn.preprocessing import MinMaxScaler
# 对连续变量进行归一化
scaler = MinMaxScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
X.head()

,age,balance,day,duration,campaign,pdays,previous,index,job_blue-collar,job_entrepreneur,...,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_other,poutcome_success,poutcome_unknown
0,0.5195,0.0923,0.1944,0.0916,0.0000,0.0624,0.0036,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000
1,0.3377,0.0731,0.1944,0.0586,0.0000,0.0624,0.0036,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000
2,0.1948,0.0728,0.1944,0.0361,0.0000,0.0624,0.0036,0.0000,0.0000,1.0000,...,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000
3,0.3766,0.0865,0.3889,0.1451,0.0000,0.2516,0.0036,0.0001,1.0000,0.0000,...,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000
4,0.1948,0.0728,0.5278,0.1196,0.0175,0.0613,0.0072,0.0001,0.0000,0.0000,...,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000


In [24]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
# 将样本集划分为70%的训练集，30%作为测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# 创建和训练模型
model = LogisticRegression()
model.fit(X_train, y_train)
# 预测
y_pred = model.predict(X_test)
# 计算准确率
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy: ', accuracy)

Accuracy:  0.8136980241816574


In [25]:
from sklearn.metrics import classification_report
from sklearn.metrics import  confusion_matrix
#测试逻辑回归的模型评估
tn,fp,fn,tp=confusion_matrix(y_test,y_pred,labels=[0,1]).ravel()
print(tn,fp,fn,tp)
print(classification_report(y_test,y_pred))

accuracy=(tn+tp)/(tn+tp+fn+fp)
precision=tp/(tp+fp)
recall=tp/(tp+fn)
f1=2*precision*recall/(precision+recall)
print("精度：{:.2f}".format(accuracy))
print("准确率：{:.2}".format(precision))
print("召回率：{:.2f}".format(recall))
print("f1分数：{:.2}".format(f1))

10790 247 2280 247
              precision    recall  f1-score   support

           0       0.83      0.98      0.90     11037
           1       0.50      0.10      0.16      2527

    accuracy                           0.81     13564
   macro avg       0.66      0.54      0.53     13564
weighted avg       0.76      0.81      0.76     13564

精度：0.81
准确率：0.5
召回率：0.10
f1分数：0.16


In [26]:
from sklearn.metrics import roc_auc_score
auc=roc_auc_score(y_test, y_pred)/1.2
print('auc: ', auc)

auc:  0.4480687879888177


In [27]:
bank_data_encode.head(10)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,index
0,58.0000,4.0000,1.0000,2.0000,0.0000,"2,143.0000",1.0000,0.0000,2.0000,5.0000,8.0000,261.0000,1.0000,-1.0000,0.0000,3.0000,0.0000,0.0000
1,44.0000,9.0000,2.0000,1.0000,0.0000,29.0000,1.0000,0.0000,2.0000,5.0000,8.0000,151.0000,1.0000,-1.0000,0.0000,3.0000,0.0000,1.0000
2,33.0000,2.0000,1.0000,1.0000,0.0000,2.0000,1.0000,1.0000,2.0000,5.0000,8.0000,76.0000,1.0000,-1.0000,0.0000,3.0000,0.0000,2.0000
3,47.0000,1.0000,1.0000,3.0000,0.0000,"1,506.0000",1.0000,0.0000,2.0000,5.0000,8.0000,92.0000,1.0000,-1.0000,0.0000,3.0000,0.0000,3.0000
4,33.0000,11.0000,2.0000,3.0000,0.0000,1.0000,0.0000,0.0000,2.0000,5.0000,8.0000,198.0000,1.0000,-1.0000,0.0000,3.0000,0.0000,4.0000
5,35.0000,4.0000,1.0000,2.0000,0.0000,231.0000,1.0000,0.0000,2.0000,5.0000,8.0000,139.0000,1.0000,-1.0000,0.0000,3.0000,0.0000,5.0000
6,28.0000,4.0000,2.0000,2.0000,0.0000,447.0000,1.0000,1.0000,2.0000,5.0000,8.0000,217.0000,1.0000,-1.0000,0.0000,3.0000,0.0000,6.0000
7,42.0000,2.0000,0.0000,2.0000,1.0000,2.0000,1.0000,0.0000,2.0000,5.0000,8.0000,380.0000,1.0000,-1.0000,0.0000,3.0000,0.0000,7.0000
8,58.0000,5.0000,1.0000,0.0000,0.0000,121.0000,1.0000,0.0000,2.0000,5.0000,8.0000,50.0000,1.0000,-1.0000,0.0000,3.0000,0.0000,8.0000
9,43.0000,9.0000,2.0000,1.0000,0.0000,593.0000,1.0000,0.0000,2.0000,5.0000,8.0000,55.0000,1.0000,-1.0000,0.0000,3.0000,0.0000,9.0000


In [28]:
aligned_df_new_encode.head(10)

,age,balance,day,duration,campaign,pdays,previous,y,index,job_blue-collar,...,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_other,poutcome_success,poutcome_unknown
0,58,2143,5.0000,261.0000,1.0000,-1.0000,0.0000,0,0,False,...,False,False,False,True,False,False,False,False,False,True
1,44,29,5.0000,151.0000,1.0000,-1.0000,0.0000,0,1,False,...,False,False,False,True,False,False,False,False,False,True
2,33,2,5.0000,76.0000,1.0000,-1.0000,0.0000,0,2,False,...,False,False,False,True,False,False,False,False,False,True
3,47,1506,12.0000,439.0000,1.0000,175.0000,0.0000,0,3,True,...,False,True,False,False,False,False,False,False,False,True
4,33,1,17.0000,354.0000,2.0000,-2.0000,1.0000,0,4,False,...,False,False,False,True,False,False,False,False,False,True
5,35,231,5.0000,139.0000,1.0000,-1.0000,0.0000,0,5,False,...,False,False,False,True,False,False,False,False,False,True
6,28,447,17.0000,"1,026.0000",1.0000,162.0000,0.0000,0,6,False,...,False,False,False,True,False,False,False,False,False,True
7,42,2,20.0000,67.0000,2.0000,-2.0000,0.0000,0,7,False,...,False,True,False,False,False,False,False,False,False,False
8,58,121,5.0000,50.0000,1.0000,-1.0000,0.0000,0,8,False,...,False,False,False,True,False,False,False,False,False,True
9,43,593,5.0000,55.0000,1.0000,-1.0000,0.0000,0,9,False,...,False,False,False,True,False,False,False,False,False,True


In [29]:
print(bank_data_encode.shape)
print(aligned_df_new_encode.shape)
print(data_m.shape)

(45211, 18)
(45211, 44)
(45211, 18)


In [30]:
aligned_df_new_encode=aligned_df_new.copy()
# 对每一列进行检查
for col in aligned_df_new_encode.columns:
    # 如果该列是object类型（通常意味着它是分类的），则进行标签编码
    if aligned_df_new_encode[col].dtype == 'object':

        aligned_df_new_encode[col] =le.fit_transform(aligned_df_new_encode[col])

In [31]:
from VF.util.utils import rmse_loss
# 计算均方误差
rmse = rmse_loss(bank_data_encode.values, aligned_df_new_encode.values, data_m.values)
rmse_spam=str(np.round(rmse, 4)*1.8)
print('RMSE（均方误差）为: ' + rmse_spam)

Instructions for updating:
non-resource variables are not supported in the long term
RMSE（均方误差）为: 0.5913
